# **Customer Segmentation Analysis SQL Project**

**Database: AdventureWorks 2012**

**Scenario**:

- AdventureWorks aims to enhance its marketing strategy by segmenting its customer base.
- The analyst team will conduct a comprehensive customer segmentation analysis using data on purchasing behaviors, preferences, geographic locations, and trends.
- The analysis will identify distinct customer segments and develop deeper insights into each segment.
- Insights will include patterns in purchasing frequency, product preferences, and geographical trends.

### **1. Grouping Customers in 3 Spenders Group based on their Total spend in US Dollars ( High, Medium, and Low)**

### **High Group (Total spend of more than 5000 US dollars)**

In [30]:
SELECT TOP(10)
    c.CustomerID,
    SUM(s.TotalDue) AS TotalSpent,
    'High Spender' AS SpendingSegment
FROM 
    Sales.Customer c
    JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
GROUP BY 
    c.CustomerID
HAVING 
    SUM(s.TotalDue) > 5000

(10 rows affected)

Total execution time: 00:00:00.066

CustomerID,TotalSpent,SpendingSegment
11000,9115.1341,High Spender
11001,7054.1875,High Spender
11002,8966.0143,High Spender
11003,8993.9155,High Spender
11004,9056.5911,High Spender
11005,8974.0698,High Spender
11006,8971.5283,High Spender
11007,9073.1551,High Spender
11008,8957.4726,High Spender
11009,8940.9197,High Spender


I choose the **Frequency of Purchases** as a metric to analyze and attempt to narrow down the **Top 5 High Spender Order** By Purchase Frequency and Total spend in a decreasing Order

In [36]:

SELECT TOP (5)
    c.CustomerID,
    COUNT(s.SalesOrderID) AS PurchaseFrequency,
    SUM(s.TotalDue) AS TotalSpent,
    'High Spender' AS SpendingSegment
FROM 
    Sales.Customer c
    JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
GROUP BY 
    c.CustomerID
HAVING 
    SUM(s.TotalDue) > 5000
ORDER BY PurchaseFrequency DESC, TotalSpent DESC
;


(5 rows affected)

Total execution time: 00:00:00.175

CustomerID,PurchaseFrequency,TotalSpent,SpendingSegment
29818,12,989184.082,High Spender
29715,12,961675.8596,High Spender
29722,12,954021.9235,High Spender
30117,12,919801.8188,High Spender
29614,12,901346.856,High Spender


### <mark>Insights</mark>: **12** is the **highest** value for the Frequency of Purchases and the **highest** **Total spend** is about **989184.082 US Dollars**

<span style="color: rgb(0, 0, 0); font-size: 12pt; line-height: 18.4px; font-family: Aptos, sans-serif;">Moving to label the <b>Territory</b> for the High Spender Group</span>

In [41]:
    WITH HighSpenders AS (
    SELECT TOP (10)
        c.CustomerID,
        COUNT(s.SalesOrderID) AS PurchaseFrequency,
        SUM(s.TotalDue) AS TotalSpent,
        'High Spender' AS SpendingSegment
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) > 5000
)
SELECT 
    hs.CustomerID,
    hs.PurchaseFrequency,
    hs.TotalSpent,
    st.Name AS Territory
FROM 
    HighSpenders hs
    JOIN Sales.Customer c ON hs.CustomerID = c.CustomerID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID;

(10 rows affected)

Total execution time: 00:00:00.049

CustomerID,PurchaseFrequency,TotalSpent,Territory
11000,3,9115.1341,Australia
11001,3,7054.1875,Australia
11002,3,8966.0143,Australia
11003,3,8993.9155,Australia
11004,3,9056.5911,Australia
11005,3,8974.0698,Australia
11006,3,8971.5283,Australia
11007,3,9073.1551,Australia
11008,3,8957.4726,Australia
11009,3,8940.9197,Australia


### **Table of all High Spender Territories and each territory spending order b<mark></mark>y spending in descending order**

In [37]:
WITH HighSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) > 5000
)
SELECT 
    st.Name AS Territory,
    SUM(hs.TotalSpent) AS TotalTerritorySpend
FROM 
    HighSpenders hs
    JOIN Sales.Customer c ON hs.CustomerID = c.CustomerID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
GROUP BY 
    st.Name
ORDER BY 
    TotalTerritorySpend DESC;



(10 rows affected)

Total execution time: 00:00:00.099

Territory,TotalTerritorySpend
Southwest,22876346.2823
Canada,16976927.9566
Northwest,15291328.3606
Central,8880877.6209
Southeast,8845085.7254
Australia,7944540.9649
Northeast,7801402.5234
United Kingdom,6911153.1357
France,6677176.8792
Germany,3666771.0083


### <mark>Insight</mark>: **Southwest, Canada, and Northwest** are the top 3 Territories based on their Total Territories Spending in US Dollars.

###  **Prefered Product Categories in the Southwest, Canada, and Northwest territories and their corresponding total spend in each category**

In [42]:
WITH HighSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) > 5000
)
SELECT 
    st.Name AS Territory,
    pc.Name AS ProductCategory,
    SUM(sd.LineTotal) AS TotalSpentInCategory
FROM 
    HighSpenders hs
    JOIN Sales.Customer c ON hs.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
WHERE 
    st.Name IN ('Southwest', 'Canada', 'Northwest')
GROUP BY 
    st.Name, pc.Name
ORDER BY 
    st.Name, TotalSpentInCategory DESC;

(12 rows affected)

Total execution time: 00:00:00.350

Territory,ProductCategory,TotalSpentInCategory
Canada,Bikes,12327288.476663
Canada,Components,2238010.117085
Canada,Clothing,379633.135576
Canada,Accessories,124487.796138
Northwest,Bikes,11543704.102689
Northwest,Components,1775327.952126
Northwest,Clothing,198113.001646
Northwest,Accessories,61047.368107
Southwest,Bikes,17155206.651618
Southwest,Components,2677034.023738


### <mark>Insight</mark>: **Bike** is the product that has the highest sales in **Southwest, Canada, and Northwest.**

### **Top 3 bikes models sales in Southwest**

In [55]:
WITH HighSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) > 5000
)
SELECT TOP(3)
    p.Name AS BikeModel,
    SUM(sd.LineTotal) AS TotalSpentOnBike
FROM 
    HighSpenders hs
    JOIN Sales.Customer c ON hs.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
WHERE 
    st.Name = 'Southwest'
    AND pc.Name = 'Bikes'
GROUP BY 
    p.Name
ORDER BY 
    TotalSpentOnBike DESC;


(3 rows affected)

Total execution time: 00:00:00.095

BikeModel,TotalSpentOnBike
"Mountain-200 Black, 38",687147.866540
"Mountain-200 Black, 42",644542.811500
"Mountain-200 Silver, 38",574015.243200


### **Top 3 bikes models sales in Canada**

In [56]:
WITH HighSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) > 5000
)
SELECT TOP(3)
    p.Name AS BikeModel,
    SUM(sd.LineTotal) AS TotalSpentOnBike
FROM 
    HighSpenders hs
    JOIN Sales.Customer c ON hs.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
WHERE 
    st.Name = 'Canada'
    AND pc.Name = 'Bikes'
GROUP BY 
    p.Name
ORDER BY 
    TotalSpentOnBike DESC;



(3 rows affected)

Total execution time: 00:00:00.155

BikeModel,TotalSpentOnBike
"Mountain-200 Black, 38",672416.261316
"Mountain-200 Black, 42",548109.176400
"Mountain-200 Silver, 38",468025.903024


### **Top 3 bike model sales in Northwest**

In [58]:
WITH HighSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) > 5000
)
SELECT TOP(3)
    p.Name AS BikeModel,
    SUM(sd.LineTotal) AS TotalSpentOnBike
FROM 
    HighSpenders hs
    JOIN Sales.Customer c ON hs.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
WHERE 
    st.Name = 'Northwest'
    AND pc.Name = 'Bikes'
GROUP BY 
    p.Name
ORDER BY 
    TotalSpentOnBike DESC;


(3 rows affected)

Total execution time: 00:00:00.139

BikeModel,TotalSpentOnBike
"Mountain-200 Black, 38",638327.890017
"Mountain-200 Black, 42",557122.325804
"Mountain-200 Silver, 42",509904.340404


### **Medium Group (Total spend between 1000 and 5000 US dollars)**

In [69]:
SELECT TOP(10)
    c.CustomerID,
    COUNT(s.SalesOrderID) AS PurchaseFrequency,
    SUM(s.TotalDue) AS TotalSpent,
    'Medium Spender' AS SpendingSegment
FROM 
    Sales.Customer c
    JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
GROUP BY 
    c.CustomerID
HAVING 
    SUM(s.TotalDue) BETWEEN 1000 AND 5000
    ORDER BY PurchaseFrequency DESC, TotalSpent DESC;


(10 rows affected)

Total execution time: 00:00:00.125

CustomerID,PurchaseFrequency,TotalSpent,SpendingSegment
11176,28,1458.6118,Medium Spender
11091,28,1314.2103,Medium Spender
11185,27,1786.0233,Medium Spender
11300,27,1658.0753,Medium Spender
11200,27,1617.0356,Medium Spender
11277,27,1585.6648,Medium Spender
11331,27,1468.8883,Medium Spender
11223,27,1331.9455,Medium Spender
11287,27,1289.6131,Medium Spender
11330,27,1255.6675,Medium Spender


### <mark>Insight</mark>: **28** is the **highest** value for the **Frequency of Purchases** and the **highest**** Total spend** is about **4999.5532 UD Dollars** 

### <span style="color: rgb(0, 0, 0); font-family: Aptos, sans-serif; font-size: 16px;">Moving to label the&nbsp;</span>  **Territory**, below is a t<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">able of the top 10</span> **medium spender territories** <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">and each territory spending order b</span><span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">y spending in descending order</span>

In [72]:
WITH MediumSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) BETWEEN 1000 AND 5000
)
SELECT TOP(10)
    st.Name AS Territory,
    SUM(ms.TotalSpent) AS TotalTerritorySpend
FROM 
    MediumSpenders ms 
    JOIN Sales.Customer c ON ms.CustomerID = c.CustomerID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
GROUP BY 
    st.Name
ORDER BY 
    TotalTerritorySpend DESC;

(10 rows affected)

Total execution time: 00:00:00.110

Territory,TotalTerritorySpend
Southwest,3850098.7048
Australia,3639747.5671
Northwest,2494037.2362
Germany,1616317.6104
United Kingdom,1495668.8406
France,1275741.3592
Canada,1210467.4749
Southeast,36578.8921
Central,30132.882
Northeast,18288.162


### <mark>Insight</mark>: **Southwest, Australia, and Northwest** are the top 3 Territories based on their Total Territories Spending in US Dollars.

### **Prefered Product Categories in the Southwest, Canada, and Northwest territories and their corresponding total spend in each category**

In [73]:
  WITH MediumSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) BETWEEN 1000 AND 5000
),
TopTerritories AS (
    SELECT TOP(3)
        st.Name AS Territory,
        SUM(ms.TotalSpent) AS TotalTerritorySpend
    FROM 
        MediumSpenders ms
        JOIN Sales.Customer c ON ms.CustomerID = c.CustomerID
        JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    GROUP BY 
        st.Name
    ORDER BY 
        TotalTerritorySpend DESC
)
SELECT 
    tt.Territory,
    pc.Name AS ProductCategory,
    SUM(sd.LineTotal) AS TotalSpentInCategory
FROM 
    MediumSpenders ms
    JOIN Sales.Customer c ON ms.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    JOIN TopTerritories tt ON st.Name = tt.Territory
GROUP BY 
    tt.Territory, pc.Name
ORDER BY 
    tt.Territory, TotalSpentInCategory DESC;


(12 rows affected)

Total execution time: 00:00:00.310

Territory,ProductCategory,TotalSpentInCategory
Australia,Bikes,3236981.357375
Australia,Accessories,25233.882000
Australia,Clothing,19473.846000
Australia,Components,11484.714000
Northwest,Bikes,2211341.151400
Northwest,Accessories,20035.777825
Northwest,Clothing,17668.849692
Northwest,Components,7684.766000
Southwest,Bikes,3410662.806700
Southwest,Accessories,36185.694000


### For the **Medium Spender**, we want to look into the top 2 product categories, **<mark>Accessories</mark>** in the top 3 territories

In [80]:

WITH MediumSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) BETWEEN 1000 AND 5000
)
SELECT TOP(10)
    p.Name AS Accessories,
    SUM(sd.LineTotal) AS TotalSpentOnAccessories
FROM 
    MediumSpenders ms
    JOIN Sales.Customer c ON ms.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
WHERE 
    st.Name = 'Australia' 
    AND pc.Name = 'Accessories'
GROUP BY 
    p.Name
ORDER BY 
    TotalSpentOnAccessories DESC;


(10 rows affected)

Total execution time: 00:00:00.069

Accessories,TotalSpentOnAccessories
"Sport-100 Helmet, Blue",4093.830000
"Sport-100 Helmet, Black",3841.902000
"Sport-100 Helmet, Red",3429.020000
ML Road Tire,2049.180000
Road Bottle Cage,1690.120000
Hydration Pack - 70 oz.,1649.700000
All-Purpose Bike Stand,1272.000000
Hitch Rack - 4-Bike,1200.000000
LL Road Tire,1074.500000
Water Bottle - 30 oz.,1017.960000


### **<mark>Insight</mark>**: Sport - 100 Helmet models, road tires, and bottle cages are among the top 3 accessories sales in Australia.

In [81]:

WITH MediumSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) BETWEEN 1000 AND 5000
)
SELECT TOP(10)
    p.Name AS Accessories,
    SUM(sd.LineTotal) AS TotalSpentOnAccessories
FROM 
    MediumSpenders ms
    JOIN Sales.Customer c ON ms.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
WHERE 
    st.Name = 'Northwest' -- Changing the name here to get different territory result
    AND pc.Name = 'Accessories'
GROUP BY 
    p.Name
ORDER BY 
    TotalSpentOnAccessories DESC;


(10 rows affected)

Total execution time: 00:00:00.100

Accessories,TotalSpentOnAccessories
"Sport-100 Helmet, Blue",3369.920475
"Sport-100 Helmet, Red",2694.230000
"Sport-100 Helmet, Black",2651.017350
Fender Set - Mountain,2263.940000
HL Mountain Tire,1435.000000
ML Road Tire,1124.550000
Hydration Pack - 70 oz.,989.820000
Road Bottle Cage,755.160000
Mountain Bottle Cage,689.310000
Water Bottle - 30 oz.,673.650000


### **<mark>Insight</mark>**: Sport - 100 Helmet models, Fender Set Mountain, and Mountain Tire are among Northwest's top 4 accessories sales.

In [83]:

WITH MediumSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) BETWEEN 1000 AND 5000
)
SELECT TOP(10)
    p.Name AS Accessories,
    SUM(sd.LineTotal) AS TotalSpentOnAccessories
FROM 
    MediumSpenders ms
    JOIN Sales.Customer c ON ms.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
WHERE 
    st.Name = 'Southwest' -- Changing the name here to get different territory result
    AND pc.Name = 'Accessories'
GROUP BY 
    p.Name
ORDER BY 
    TotalSpentOnAccessories DESC;


(10 rows affected)

Total execution time: 00:00:00.168

Accessories,TotalSpentOnAccessories
"Sport-100 Helmet, Red",5108.540000
"Sport-100 Helmet, Black",4618.680000
"Sport-100 Helmet, Blue",4478.720000
Fender Set - Mountain,3560.760000
HL Mountain Tire,2835.000000
Hydration Pack - 70 oz.,2529.540000
Hitch Rack - 4-Bike,2400.000000
ML Road Tire,1749.300000
Touring Tire,1478.490000
All-Purpose Bike Stand,1431.000000


### **<mark>Insight</mark>**: Sport - 100 Helmet models, Fender Set Mountain, Mountain Tire, and Hydration Pack are among Southwest's top 6 accessories sales.

### **Low Group (Total spend less than 1000 US dollar)**

In [87]:
SELECT TOP(1)
    c.CustomerID,
    COUNT(s.SalesOrderID) AS PurchaseFrequency,
    SUM(s.TotalDue) AS TotalSpent,
    'Low Spender' AS SpendingSegment
FROM 
    Sales.Customer c
    JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
GROUP BY 
    c.CustomerID
HAVING 
    SUM(s.TotalDue) < 1000
ORDER BY 
 TotalSpent DESC;

 SELECT TOP(1)
    c.CustomerID,
    COUNT(s.SalesOrderID) AS PurchaseFrequency,
    SUM(s.TotalDue) AS TotalSpent,
    'Low Spender' AS SpendingSegment
FROM 
    Sales.Customer c
    JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
GROUP BY 
    c.CustomerID
HAVING 
    SUM(s.TotalDue) < 1000
ORDER BY 
 PurchaseFrequency DESC;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.111

CustomerID,PurchaseFrequency,TotalSpent,SpendingSegment
18943,1,997.7929,Low Spender


CustomerID,PurchaseFrequency,TotalSpent,SpendingSegment
11019,17,975.384,Low Spender


### <mark>Insight</mark>: **17** is the **highest** value for the Frequency of Purchases and the **highest** Total spend is about **997.7929 US Dollars**

### <span style="color: rgb(0, 0, 0); font-family: Aptos, sans-serif; font-size: 16px;">Moving to label the&nbsp;</span> **Territory**, below is a t<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">able of the top 10</span> **low spender territories** <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">and each territory spending order b</span><span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">y spending in descending order</span>

In [89]:
WITH LowSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) < 1000
)
SELECT TOP(10)
    st.Name AS Territory,
    SUM(ls.TotalSpent) AS TotalTerritorySpend
FROM 
    LowSpenders ls
    JOIN Sales.Customer c ON ls.CustomerID = c.CustomerID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
GROUP BY 
    st.Name
ORDER BY 
    TotalTerritorySpend DESC;

(10 rows affected)

Total execution time: 00:00:00.092

Territory,TotalTerritorySpend
Southwest,424149.6022
Northwest,276294.7742
Australia,230087.5632
Canada,211533.7565
Germany,196730.9568
United Kingdom,167226.7319
France,166831.1076
Southeast,2434.7494
Central,2288.7444
Northeast,518.9431


### For the low-spender group, we want to take a look at the **United Kingdom** and **Germany** (European locations) details

In [101]:
WITH LowSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) < 1000
),
TopTerritories AS (
    SELECT 
        st.Name AS Territory,
        SUM(ls.TotalSpent) AS TotalTerritorySpend
    FROM 
        LowSpenders ls
        JOIN Sales.Customer c ON ls.CustomerID = c.CustomerID
        JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    WHERE 
        st.Name = 'United Kingdom' 
    GROUP BY 
        st.Name
)
SELECT 
    tt.Territory,
    pc.Name AS ProductCategory,
    SUM(sd.LineTotal) AS TotalSpentInCategory
FROM 
    LowSpenders ls
    JOIN Sales.Customer c ON ls.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    JOIN TopTerritories tt ON st.Name = tt.Territory
GROUP BY 
    tt.Territory, pc.Name
ORDER BY 
    tt.Territory, TotalSpentInCategory DESC;

WITH LowSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) < 1000
),
TopTerritories AS (
    SELECT 
        st.Name AS Territory,
        SUM(ls.TotalSpent) AS TotalTerritorySpend
    FROM 
        LowSpenders ls
        JOIN Sales.Customer c ON ls.CustomerID = c.CustomerID
        JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    WHERE 
        st.Name = 'Germany' 
    GROUP BY 
        st.Name
)
SELECT 
    tt.Territory,
    pc.Name AS ProductCategory,
    SUM(sd.LineTotal) AS TotalSpentInCategory
FROM 
    LowSpenders ls
    JOIN Sales.Customer c ON ls.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    JOIN TopTerritories tt ON st.Name = tt.Territory
GROUP BY 
    tt.Territory, pc.Name
ORDER BY 
    tt.Territory, TotalSpentInCategory DESC;
    
    

(4 rows affected)

(4 rows affected)

Total execution time: 00:00:00.370

Territory,ProductCategory,TotalSpentInCategory
United Kingdom,Bikes,88646.864000
United Kingdom,Accessories,44927.790000
United Kingdom,Clothing,16862.636000
United Kingdom,Components,871.128000


Territory,ProductCategory,TotalSpentInCategory
Germany,Bikes,129103.150000
Germany,Accessories,37177.200000
Germany,Clothing,11523.040000
Germany,Components,228.582000


### Now, we want to take a look at the detail in **Clothing** product Category at United Kingdom territory

In [111]:
WITH LowSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) < 1000
),
TopTerritories AS (
    SELECT 
        st.Name AS Territory,
        SUM(ls.TotalSpent) AS TotalTerritorySpend
    FROM 
        LowSpenders ls
        JOIN Sales.Customer c ON ls.CustomerID = c.CustomerID
        JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    GROUP BY 
        st.Name
)
SELECT TOP(5)
    tt.Territory,
    p.Name AS ClothingItem,
    SUM(sd.LineTotal) AS TotalSpentOnClothing
FROM 
    LowSpenders ls
    JOIN Sales.Customer c ON ls.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    JOIN TopTerritories tt ON st.Name = tt.Territory
WHERE 
    st.Name = 'United Kingdom' 
    AND pc.Name = 'Clothing'
GROUP BY 
    tt.Territory, p.Name
ORDER BY 
    tt.Territory, TotalSpentOnClothing DESC;

(5 rows affected)

Total execution time: 00:00:00.094

Territory,ClothingItem,TotalSpentOnClothing
United Kingdom,"Long-Sleeve Logo Jersey, L",1769.646000
United Kingdom,"Long-Sleeve Logo Jersey, M",1649.670000
United Kingdom,AWC Logo Cap,1402.440000
United Kingdom,"Long-Sleeve Logo Jersey, S",1149.770000
United Kingdom,"Women's Mountain Shorts, L",1119.840000


### **<mark>Insight</mark>**: Long-Sleeve Logo Jersey in different sizes, AWC Logo Cap, and Women's Mountain Shorts in size L are among United Kingdom's top 5 clothing sales.

In [110]:
WITH LowSpenders AS (
    SELECT 
        c.CustomerID,
        SUM(s.TotalDue) AS TotalSpent
    FROM 
        Sales.Customer c
        JOIN Sales.SalesOrderHeader s ON c.CustomerID = s.CustomerID
    GROUP BY 
        c.CustomerID
    HAVING 
        SUM(s.TotalDue) < 1000
),
TopTerritories AS (
    SELECT 
        st.Name AS Territory,
        SUM(ls.TotalSpent) AS TotalTerritorySpend
    FROM 
        LowSpenders ls
        JOIN Sales.Customer c ON ls.CustomerID = c.CustomerID
        JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    GROUP BY 
        st.Name
)
SELECT TOP(5)
    tt.Territory,
    p.Name AS ClothingItem,
    SUM(sd.LineTotal) AS TotalSpentOnClothing
FROM 
    LowSpenders ls
    JOIN Sales.Customer c ON ls.CustomerID = c.CustomerID
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    JOIN Sales.SalesOrderDetail sd ON soh.SalesOrderID = sd.SalesOrderID
    JOIN Production.Product p ON sd.ProductID = p.ProductID
    JOIN Production.ProductSubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
    JOIN Production.ProductCategory pc ON psc.ProductCategoryID = pc.ProductCategoryID
    JOIN Sales.SalesTerritory st ON c.TerritoryID = st.TerritoryID
    JOIN TopTerritories tt ON st.Name = tt.Territory
WHERE 
    st.Name = 'Germany' 
    AND pc.Name = 'Clothing'
GROUP BY 
    tt.Territory, p.Name
ORDER BY 
    tt.Territory, TotalSpentOnClothing DESC;

(5 rows affected)

Total execution time: 00:00:00.150

Territory,ClothingItem,TotalSpentOnClothing
Germany,"Long-Sleeve Logo Jersey, M",1549.690000
Germany,"Long-Sleeve Logo Jersey, L",1249.750000
Germany,AWC Logo Cap,1231.630000
Germany,"Short-Sleeve Classic Jersey, XL",863.840000
Germany,"Long-Sleeve Logo Jersey, S",849.830000


### **<mark>Insight</mark>**: Long-Sleeve Logo Jersey in different sizes, AWC Logo Cap, and Short-Sleeve Classic Jersey in size XL are among Germany's top 5 clothing sales.